# Beta-VAE

In [1]:
# coding=utf-8
# Copyright 2018 The DisentanglementLib Authors.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Implementation of the disentanglement metric from the BetaVAE paper.

Based on "beta-VAE: Learning Basic Visual Concepts with a Constrained
Variational Framework" (https://openreview.net/forum?id=Sy2fzU9gl).
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from absl import logging
import numpy as np
from sklearn import linear_model
import gin.tf

def compute_beta_vae_sklearn(ground_truth_labels,
                             representations,
                             random_state,
                             num_variables,
                             artifact_dir=None,
                             batch_size=gin.REQUIRED,
                             num_train=gin.REQUIRED,
                             num_eval=gin.REQUIRED):
  """Computes the BetaVAE disentanglement metric using scikit-learn.

  Args:
    ground_truth_data: GroundTruthData to be sampled from.
    representation_function: Function that takes observations as input and
      outputs a dim_representation sized representation for each observation.
    random_state: Numpy random state used for randomness.
    artifact_dir: Optional path to directory where artifacts can be saved.
    batch_size: Number of points to be used to compute the training_sample.
    num_train: Number of points used for training.
    num_eval: Number of points used for evaluation.

  Returns:
    Dictionary with scores:
      train_accuracy: Accuracy on training set.
      eval_accuracy: Accuracy on evaluation set.
  """
  del artifact_dir
  train_points, train_labels = _generate_training_batch(
      ground_truth_labels, representations, batch_size, num_train,
      random_state, num_variables)

  model = linear_model.LogisticRegression(random_state=random_state)
  model.fit(train_points, train_labels)

  logging.info("Evaluate training set accuracy.")
  train_accuracy = model.score(train_points, train_labels)
  train_accuracy = np.mean(model.predict(train_points) == train_labels)

  eval_points, eval_labels = _generate_training_batch(
      ground_truth_labels, representations, batch_size, num_eval,
      random_state,num_variables)

  logging.info("Evaluate evaluation set accuracy.")
  eval_accuracy = model.score(eval_points, eval_labels)
  logging.info("Evaluation set accuracy: %.2g", eval_accuracy)
  scores_dict = {}
  scores_dict["train_accuracy"] = train_accuracy
  scores_dict["eval_accuracy"] = eval_accuracy
  return scores_dict


def _generate_training_batch(ground_truth_labels, representations,
                             batch_size, num_points, random_state,num_variables):
  """Sample a set of training samples based on a batch of ground-truth data.

  Args:
    ground_truth_data: GroundTruthData to be sampled from.
    representation_function: Function that takes observations as input and
      outputs a dim_representation sized representation for each observation.
    batch_size: Number of points to be used to compute the training_sample.
    num_points: Number of points to be sampled for training set.
    random_state: Numpy random state used for randomness.

  Returns:
    points: (num_points, dim_representation)-sized numpy array with training set
      features.
    labels: (num_points)-sized numpy array with training set labels.
  """
  points = None  # Dimensionality depends on the representation function.
  labels = np.zeros(num_points, dtype=np.int64)
  for i in range(num_points):
    labels[i], feature_vector = _generate_training_sample(
        ground_truth_labels, representations, batch_size, random_state,num_variables)
    if points is None:
      points = np.zeros((num_points, feature_vector.shape[0]))
    points[i, :] = feature_vector
  return points, labels

#start and end points for each property
def _sample_factors_min_max(index):
  #size fixed
  if index == 0:
    min = 3
    max = 6
  #texture fixed
  if index == 1:
    min = 6
    max = 66
  #shape fixed
  if index == 2:
    min = 66
    max = 70
  return min,max

#get a representation that fits the fixed label
def _getRandomRepresentation(labels, datapoints, label, randomState):
  start = randomState.randint(len(datapoints))
  start_pos = start
  while start < len(labels) and labels[start][label] != 1:
    start = start + 1
    if start == len(labels):
      start = 0
  return datapoints[start], start

#sample fixed factors
def _sample_factors(ground_truth_labels, representations, batch_size, random_state,index):
  min ,max = _sample_factors_min_max(index)
  representations1 = []
  representations2 = []
  for i in range(batch_size):
    randomLabel = random_state.randint(min,max)
    representation1, returnIndex = _getRandomRepresentation(ground_truth_labels, representations,randomLabel, random_state)
    representation2, returnIndex = _getRandomRepresentation(ground_truth_labels, representations,randomLabel, random_state)
    representations1.append(representation1)
    representations2.append(representation2)
  return np.array(representations1), np.array(representations2)


def _generate_training_sample(ground_truth_labels, representations,
                              batch_size, random_state, num_variables):
  """Sample a single training sample based on a mini-batch of ground-truth data.

  Args:
    ground_truth_data: GroundTruthData to be sampled from.
    representation_function: Function that takes observation as input and
      outputs a representation.
    batch_size: Number of points to be used to compute the training_sample
    random_state: Numpy random state used for randomness.

  Returns:
    index: Index of coordinate to be used.
    feature_vector: Feature vector of training sample.
  """
  # Select random coordinate to keep fixed.
  index = random_state.randint(num_variables)
  # Sample two mini batches of latent variables.
  representation1, representation2 = _sample_factors(ground_truth_labels, representations,batch_size, random_state, index)
  # Ensure sampled coordinate is the same across pairs of samples.

  # Compute the feature vector based on differences in representation.
  feature_vector = np.mean(np.abs(representation1 - representation2), axis=0)
  return index, feature_vector

ModuleNotFoundError: No module named 'gin'

In [ ]:
#Execute with:
random_state = np.random.RandomState(10)
num_variables = 3
path_name "YOUR_PATH"
labels = np.load(path_name + "/binded_properties.npy")
slots =  np.load(path_name + "/binded_slots.npy")
batch_size = 10
num_train = 1000
num_test = 1000

compute_beta_vae_sklearn(labels,slots,random_state,num_variables,artifact_dir=None,batch_size=batch_size,num_train=num_train,num_eval=num_test)

# DCI

In [ ]:
#argmax over each property
def transformLabel(label):
  newLabel = []
  newLabel.append(np.argmax(label[3:6])) 
  newLabel.append(np.argmax(label[6:71-5]))
  newLabel.append(np.argmax(label[71-5:71-1]))
  return np.array(newLabel)


def _generate_batch_factor_code(slots, labels,
                               num_points,random_state, batch_size):
  """Sample a single training sample based on a mini-batch of ground-truth data.

  Args:
    ground_truth_data: GroundTruthData to be sampled from.
    representation_function: Function that takes observation as input and
      outputs a representation.
    num_points: Number of points to sample.
    random_state: Numpy random state used for randomness.
    batch_size: Batchsize to sample points.

  Returns:
    representations: Codes (num_codes, num_points)-np array.
    factors: Factors generating the codes (num_factors, num_points)-np array.
  """
  representations = None
  factors = None
  i = 0
  while i < num_points:
    randomPoint = random_state.randint(len(slots))
    slot_batched = slots[randomPoint]
    label_batched = labels[randomPoint]
    label_batched = transformLabel(label_batched)
    current_factors = np.array([label_batched])
    current_slot = np.array([slot_batched])
    if i == 0:
      factors = current_factors
      representations = current_slot
    else:
      factors = np.vstack((factors, current_factors))
      representations = np.vstack((representations,current_slot))
    i = i + 1
  return np.transpose(representations), np.transpose(factors)

In [ ]:
# coding=utf-8
# Copyright 2018 The DisentanglementLib Authors.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Implementation of Disentanglement, Completeness and Informativeness.

Based on "A Framework for the Quantitative Evaluation of Disentangled
Representations" (https://openreview.net/forum?id=By-7dz-AZ).
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from absl import logging
import numpy as np
import scipy
from sklearn import ensemble
import gin.tf
from sklearn.tree import DecisionTreeClassifier


def compute_dci(slots, labels, random_state,
                num_train,
                num_test,
                batch_size=16,
                artifact_dir=None):
  """Computes the DCI scores according to Sec 2.

  Args:
    ground_truth_data: GroundTruthData to be sampled from.
    representation_function: Function that takes observations as input and
      outputs a dim_representation sized representation for each observation.
    random_state: Numpy random state used for randomness.
    artifact_dir: Optional path to directory where artifacts can be saved.
    num_train: Number of points used for training.
    num_test: Number of points used for testing.
    batch_size: Batch size for sampling.

  Returns:
    Dictionary with average disentanglement score, completeness and
      informativeness (train and test).
  """
  del artifact_dir
  logging.info("Generating training set.")
  # mus_train are of shape [num_codes, num_train], while ys_train are of shape
  # [num_factors, num_train].
  mus_train, ys_train = _generate_batch_factor_code(
      slots, labels, num_train,
      random_state, batch_size)
  assert mus_train.shape[1] == num_train
  assert ys_train.shape[1] == num_train
  mus_test, ys_test = _generate_batch_factor_code(
      slots, labels, num_test,
      random_state, batch_size)
  print(ys_test.shape)
  scores = _compute_dci(mus_train, ys_train, mus_test, ys_test)
  return scores


def _compute_dci(mus_train, ys_train, mus_test, ys_test):
  """Computes score based on both training and testing codes and factors."""
  scores = {}
  importance_matrix, train_err, test_err = compute_importance_gbt(
      mus_train, ys_train, mus_test, ys_test)
  assert importance_matrix.shape[0] == mus_train.shape[0]
  assert importance_matrix.shape[1] == ys_train.shape[0]
  scores["informativeness_train"] = train_err
  scores["informativeness_test"] = test_err
  scores["disentanglement"] = disentanglement(importance_matrix)
  scores["completeness"] = completeness(importance_matrix)
  return scores

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier


def compute_importance_gbt(x_train, y_train, x_test, y_test):
  """Compute importance based on gradient boosted trees."""
  num_factors = y_train.shape[0]
  num_codes = x_train.shape[0]
  importance_matrix = np.zeros(shape=[num_codes, num_factors],
                               dtype=np.float64)
  train_loss = []
  test_loss = []
  for i in range(num_factors):
    print(i)
    model = ensemble.GradientBoostingClassifier()
    #model = DecisionTreeClassifier()
    #model = RandomForestClassifier()
    model.fit(x_train.T, y_train[i, :])
    importance_matrix[:, i] = np.abs(model.feature_importances_)
    train_loss.append(np.mean(model.predict(x_train.T) == y_train[i, :]))
    test_loss.append(np.mean(model.predict(x_test.T) == y_test[i, :]))
  return importance_matrix, np.mean(train_loss), np.mean(test_loss)


def disentanglement_per_code(importance_matrix):
  """Compute disentanglement score of each code."""
  # importance_matrix is of shape [num_codes, num_factors].
  return 1. - scipy.stats.entropy(importance_matrix.T + 1e-11,
                                  base=importance_matrix.shape[1])


def disentanglement(importance_matrix):
  """Compute the disentanglement score of the representation."""
  per_code = disentanglement_per_code(importance_matrix)
  if importance_matrix.sum() == 0.:
    importance_matrix = np.ones_like(importance_matrix)
  code_importance = importance_matrix.sum(axis=1) / importance_matrix.sum()

  return np.sum(per_code*code_importance)


def completeness_per_factor(importance_matrix):
  """Compute completeness of each factor."""
  # importance_matrix is of shape [num_codes, num_factors].
  return 1. - scipy.stats.entropy(importance_matrix + 1e-11,
                                  base=importance_matrix.shape[0])


def completeness(importance_matrix):
  """"Compute completeness of the representation."""
  per_factor = completeness_per_factor(importance_matrix)
  if importance_matrix.sum() == 0.:
    importance_matrix = np.ones_like(importance_matrix)
  factor_importance = importance_matrix.sum(axis=0) / importance_matrix.sum()
  return np.sum(per_factor*factor_importance)

In [ ]:
#Execute with:
random_state = np.random.RandomState(10)
num_variables = 3
path_name "YOUR_PATH"
labels = np.load(path_name + "/binded_properties.npy")
slots =  np.load(path_name + "/binded_slots.npy")
batch_size = 10
num_train = 1000
num_test = 1000

compute_dci(slots, labels, rando_state,num_train,num_test,batch_size=batch_size,artifact_dir=None)